In [85]:
import os
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import warnings

import utils

In [24]:
scoring = {
    'pass_yd': .04,
    'pass_td': 4,
    'pass_2pt': 2,
    'pass_int': -1,
    'rush_yd': .1,
    'rush_td': 6,
    'rush_2pt': 2,
    'rec_rcpt': .5,
    'rec_yd': .1,
    'rec_td': 6,
    'rec_2pt': 2,
    'remainder': 1
}

In [72]:

# def sample_fp(df):
#     '''
#     takes a long dataframe with id column called sleeper_id, 1 kind of stat per row, and a value column
#     the input dataframe should have stats from multiple sources
#     this function will sample 1 stat of each type per player and compute the fpts
#     '''
#     df = df.groupby(["sleeper_id", "stat"]).apply(lambda x: x.sample(1)).reset_index(drop=True)
#     # compute the new fpts for each sleeper_id
#     df["fpts"] = df.apply(lambda x: x["value"]*scoring[x["stat"]], axis=1)
#     # summarize the fpts for each sleeper_id
#     df = df.groupby("sleeper_id").agg({"fpts": "sum"}).reset_index()
#     return df

def sample_fp(df):
    '''
    takes a long dataframe with id column called sleeper_id, 1 kind of stat per row, and a value column
    the input dataframe should have stats from multiple sources
    this function will sample 1 stat of each type per player and compute the fpts
    '''
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=DeprecationWarning)
        df = df.groupby(["sleeper_id", "stat"]).apply(lambda x: x.sample(1))[df.columns].reset_index(drop=True)
    # Compute the new fpts for each sleeper_id
    df["fpts"] = df.apply(lambda x: x["value"] * scoring[x["stat"]], axis=1)
    # Summarize the fpts for each sleeper_id
    df = df.groupby("sleeper_id").agg({"fpts": "sum"}).reset_index()
    return df

In [88]:

df_sleeper = pd.read_csv("data/sleeper/all_players.csv")

dict_projections = utils.load_projections("data/projections")
list(dict_projections.keys())
dict_projections["K"] = pd.read_csv("data/projections/K_projections.csv")


In [73]:


df_qb = dict_projections["QB"].loc[:, ["sleeper_id", "pass_yd", "pass_td", "pass_int", "rush_yd", "rush_td", "fpts"]]
df_qb = df_qb.apply(pd.to_numeric, errors='coerce')
df_qb["remainder"] = df_qb["fpts"] - (df_qb["pass_yd"]*scoring["pass_yd"] + \
    df_qb["pass_td"]*scoring["pass_td"] + df_qb["pass_int"]*scoring["pass_int"] + \
        df_qb["rush_yd"]*scoring["rush_yd"] + df_qb["rush_td"]*scoring["rush_td"])
df_qb = df_qb.drop(columns=["fpts"])
# pivot longer
df_qb = df_qb.melt(id_vars=["sleeper_id"], var_name="stat", value_name="value")
df_qb_samples = pd.concat([sample_fp(df_qb) for _ in range(5000)])

# re-add the player name, team, and position
df_qb_samples = df_qb_samples.merge(df_sleeper.loc[:, ["sleeper_id", "full_name", "team", "position"]], on="sleeper_id")

df_qb_samples.to_csv("data/proj_bootstrap/qb_proj_resamples.csv", index=False)

In [74]:

df_rb = dict_projections["RB"].loc[:, ["sleeper_id", "rush_yd", "rush_td", "rec_rcpt", "rec_yd", "rec_td", "fpts"]]
df_rb = df_rb.apply(pd.to_numeric, errors='coerce')
df_rb["remainder"] = df_rb["fpts"] - (df_rb["rush_yd"]*scoring["rush_yd"] + \
    df_rb["rush_td"]*scoring["rush_td"] + df_rb["rec_rcpt"]*scoring["rec_rcpt"] + \
        df_rb["rec_yd"]*scoring["rec_yd"] + df_rb["rec_td"]*scoring["rec_td"])
df_rb = df_rb.drop(columns=["fpts"])
# pivot longer
df_rb = df_rb.melt(id_vars=["sleeper_id"], var_name="stat", value_name="value")
df_rb_samples = pd.concat([sample_fp(df_rb) for _ in range(5000)])

# re-add the player name, team, and position
df_rb_samples = df_rb_samples.merge(df_sleeper.loc[:, ["sleeper_id", "full_name", "team", "position"]], on="sleeper_id")

df_rb_samples.to_csv("data/proj_bootstrap/rb_proj_resamples.csv", index=False)


In [75]:
df_wr = dict_projections["WR"].loc[:, ["sleeper_id", "rush_yd", "rush_td", "rec_rcpt", "rec_yd", "rec_td", "fpts"]]
df_wr = df_wr.apply(pd.to_numeric, errors='coerce')
df_wr["remainder"] = df_wr["fpts"] - (df_wr["rush_yd"]*scoring["rush_yd"] + \
    df_wr["rush_td"]*scoring["rush_td"] + df_wr["rec_rcpt"]*scoring["rec_rcpt"] + \
        df_wr["rec_yd"]*scoring["rec_yd"] + df_wr["rec_td"]*scoring["rec_td"])
df_wr = df_wr.drop(columns=["fpts"])
# pivot longer
df_wr = df_wr.melt(id_vars=["sleeper_id"], var_name="stat", value_name="value")
df_wr_samples = pd.concat([sample_fp(df_wr) for _ in range(5000)])

# re-add the player name, team, and position
df_wr_samples = df_wr_samples.merge(df_sleeper.loc[:, ["sleeper_id", "full_name", "team", "position"]], on="sleeper_id")

df_wr_samples.to_csv("data/proj_bootstrap/wr_proj_resamples.csv", index=False)


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x110a24c40>>
Traceback (most recent call last):
  File "/Users/ashish/files/fun_projects/fantasy_football_2024/venv/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [77]:
df_te = dict_projections["TE"].loc[:, ["sleeper_id", "rush_yd", "rush_td", "rec_rcpt", "rec_yd", "rec_td", "fpts"]]
df_te = df_te.apply(pd.to_numeric, errors='coerce')
df_te["remainder"] = df_te["fpts"] - (df_te["rush_yd"]*scoring["rush_yd"] + \
    df_te["rush_td"]*scoring["rush_td"] + df_te["rec_rcpt"]*scoring["rec_rcpt"] + \
        df_te["rec_yd"]*scoring["rec_yd"] + df_te["rec_td"]*scoring["rec_td"])
df_te = df_te.drop(columns=["fpts"])
# pivot longer
df_te = df_te.melt(id_vars=["sleeper_id"], var_name="stat", value_name="value")
df_te_samples = pd.concat([sample_fp(df_te) for _ in range(5000)])

# re-add the player name, team, and position
df_te_samples = df_te_samples.merge(df_sleeper.loc[:, ["sleeper_id", "full_name", "team", "position"]], on="sleeper_id")

df_te_samples.to_csv("data/proj_bootstrap/te_proj_resamples.csv", index=False)


In [89]:
df_k = dict_projections["K"].loc[:, ["sleeper_id", "fpts"]]
df_k = df_k.apply(pd.to_numeric, errors='coerce')
df_k_samples = pd.concat([sample_fp(df_k) for _ in range(5000)])

# re-add the player name, team, and position
df_k_samples = df_k_samples.merge(df_sleeper.loc[:, ["sleeper_id", "full_name", "team", "position"]], on="sleeper_id")

df_k_samples.to_csv("data/proj_bootstrap/k_proj_resamples.csv", index=False)

KeyError: 'stat'

In [ ]:
df_def = dict_projections["DEF"].loc[:, ["sleeper_id", "fpts"]]
df_def = df_def.apply(pd.to_numeric, errors='coerce')
df_def_samples = pd.concat([sample_fp(df_def) for _ in range(5000)])

# re-add the player name, team, and position
df_def_samples = df_def_samples.merge(df_sleeper.loc[:, ["sleeper_id", "full_name", "team", "position"]], on="sleeper_id")

df_def_samples.to_csv("data/proj_bootstrap/def_proj_resamples.csv", index=False)
